In [1]:
from nowcasting_dataset.datamodule import NowcastingDataModule
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import numcodecs
import gcsfs
from typing import List
import io

import logging
logging.basicConfig()
logger = logging.getLogger('nowcasting_dataset')
logger.setLevel(logging.DEBUG)

## Load Zarr batches

In [2]:
filename = 'gs://solar-pv-nowcasting-data/prepared_ML_training_data/testing.zarr'

In [3]:
%%time
ds = xr.open_dataset(filename, engine='zarr', chunks={})

CPU times: user 20.9 s, sys: 899 ms, total: 21.8 s
Wall time: 29.5 s


In [4]:
ds

<xarray.Dataset>
Dimensions:               (example: 974880, nwp_variable: 10, nwp_x: 2, nwp_y: 2, sat_variable: 12, sat_x: 32, sat_y: 32, time: 19)
Coordinates:
  * example               (example) int32 0 1 2 3 ... 974877 974878 974879
    init_time             (example) datetime64[ns] dask.array<chunksize=(5442,), meta=np.ndarray>
  * nwp_variable          (nwp_variable) <U5 't' 'dswrf' 'prate' ... 'mcc' 'hcc'
  * nwp_x                 (nwp_x) int32 0 1
  * nwp_y                 (nwp_y) int32 0 1
  * sat_variable          (sat_variable) <U6 'HRV' 'IR_016' ... 'WV_073'
  * sat_x                 (sat_x) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * sat_y                 (sat_y) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * time                  (time) int32 0 1 2 3 4 5 6 7 ... 12 13 14 15 16 17 18
Data variables:
    nwp                   (example, nwp_variable, time, nwp_y, nwp_x) float32 dask.array<chunksize=(32, 10, 19, 2, 2), meta=np.ndarray>
    nwp_time_coords       (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    nwp_x_coords          (example, nwp_x) float32 dask.array<chunksize=(32, 2), meta=np.ndarray>
    nwp_y_coords          (example, nwp_y) float32 dask.array<chunksize=(32, 2), meta=np.ndarray>
    pv_system_id          (example) int32 dask.array<chunksize=(32,), meta=np.ndarray>
    pv_system_row_number  (example) int32 dask.array<chunksize=(32,), meta=np.ndarray>
    pv_yield              (example, time) float32 dask.array<chunksize=(32, 19), meta=np.ndarray>
    pv_yield_time_coords  (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    sat_data              (example, time, sat_y, sat_x, sat_variable) float32 dask.array<chunksize=(32, 19, 32, 32, 12), meta=np.ndarray>
    sat_time_coords       (example, time) datetime64[ns] dask.array<chunksize=(32, 19), meta=np.ndarray>
    sat_x_coords          (example, sat_x) int32 dask.array<chunksize=(32, 32), meta=np.ndarray>
    sat_y_coords          (example, sat_y) int32 dask.array<chunksize=(32, 32), meta=np.ndarray>

In [440]:
%%time
batch = ds.sel(example=slice(32, 61)).load()

CPU times: user 213 ms, sys: 15.8 ms, total: 229 ms
Wall time: 334 ms


In [6]:
batch

<xarray.Dataset>
Dimensions:               (example: 32, nwp_variable: 10, nwp_x: 2, nwp_y: 2, sat_variable: 12, sat_x: 32, sat_y: 32, time: 19)
Coordinates:
  * example               (example) int32 0 1 2 3 4 5 6 ... 25 26 27 28 29 30 31
    init_time             (example) datetime64[ns] 2018-08-13T09:00:00 ... 20...
  * nwp_variable          (nwp_variable) <U5 't' 'dswrf' 'prate' ... 'mcc' 'hcc'
  * nwp_x                 (nwp_x) int32 0 1
  * nwp_y                 (nwp_y) int32 0 1
  * sat_variable          (sat_variable) <U6 'HRV' 'IR_016' ... 'WV_073'
  * sat_x                 (sat_x) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * sat_y                 (sat_y) int32 0 1 2 3 4 5 6 7 ... 25 26 27 28 29 30 31
  * time                  (time) int32 0 1 2 3 4 5 6 7 ... 12 13 14 15 16 17 18
Data variables:
    nwp                   (example, nwp_variable, time, nwp_y, nwp_x) float32 ...
    nwp_time_coords       (example, time) datetime64[ns] 2018-08-13T10:55:00 ...
    nwp_x_coords          (example, nwp_x) float32 5.02e+05 5.04e+05 ... 2e+05
    nwp_y_coords          (example, nwp_y) float32 1.64e+05 1.62e+05 ... 5.2e+04
    pv_system_id          (example) int32 5182 9785 49888 ... 32548 4275 58831
    pv_system_row_number  (example) int32 127 224 676 18 612 ... 716 421 75 771
    pv_yield              (example, time) float32 0.7409 0.7073 ... 0.0 0.0
    pv_yield_time_coords  (example, time) datetime64[ns] 2018-08-13T10:55:00 ...
    sat_data              (example, time, sat_y, sat_x, sat_variable) float32 ...
    sat_time_coords       (example, time) datetime64[ns] 2018-08-13T10:55:00 ...
    sat_x_coords          (example, sat_x) int32 472000 474000 ... 228000 230000
    sat_y_coords          (example, sat_y) int32 194000 192000 ... 24000 22000

## Writing NETCDF3_64BIT to Google Cloud, without saving to local filesystem first

`Dataset.to_netcdf()` always uses the `scipy` engine when the `filename` is not a (string) filename on the local filesystem.

In [10]:

# write single batch to GCS
gcs = gcsfs.GCSFileSystem(access='read_write')

In [48]:
%%time

ENGINE = 'scipy'
FORMAT = 'NETCDF3_64BIT'

nc_bytes = batch.to_netcdf(engine=ENGINE, format=FORMAT) #', format='NETCDF4') #, encoding=encoding)  # TODO Get compression working!

CPU times: user 26.3 ms, sys: 23.8 ms, total: 50.1 ms
Wall time: 48.7 ms


In [49]:
%%time
with gcs.open('gs://solar-pv-nowcasting-data/prepared_ML_training_data/test.nc', mode='wb') as output_file:
    output_file.write(nc_bytes)

In [182]:
%%time
with gcs.open('gs://solar-pv-nowcasting-data/prepared_ML_training_data/test.nc', mode='rb') as file:
    file_bytes = file.read()
    
with io.BytesIO(file_bytes) as file:
    batch_loaded = xr.load_dataset(file, engine=ENGINE, format=FORMAT)

CPU times: user 64 ms, sys: 21.2 ms, total: 85.2 ms
Wall time: 190 ms


## NETCDF4 on local filesystem

In [443]:
local_filename = 'test.nc'

encoding = {
    name: {'zlib': True} #{'compression': 'lzf'}
    for name in batch.data_vars}

In [444]:
%%time
batch.to_netcdf(local_filename, engine='h5netcdf', mode='w', encoding=encoding)

CPU times: user 555 ms, sys: 6.54 ms, total: 561 ms
Wall time: 599 ms


In [445]:
!ls -alh $local_filename

-rw-rw-r-- 1 jack jack 11M Jul 21 17:24 test.nc


Filesize:  29 MB uncompressed; 12 MB with `zlib: True`

In [447]:
%%time
batch_loaded = xr.load_dataset('test.nc', engine='netcdf4')

CPU times: user 205 ms, sys: 3.55 ms, total: 209 ms
Wall time: 208 ms


## NETCDF4 from GCS

In [448]:
%%time
gcs.put(local_filename, 'gs://solar-pv-nowcasting-data/prepared_ML_training_data/test_nc4.nc')

CPU times: user 27.6 ms, sys: 16.8 ms, total: 44.4 ms
Wall time: 247 ms


[None]

In [449]:
%%time
with gcs.open('gs://solar-pv-nowcasting-data/prepared_ML_training_data/test_nc4.nc', mode='rb') as file:
    file_bytes = file.read()

CPU times: user 27.5 ms, sys: 0 ns, total: 27.5 ms
Wall time: 111 ms


In [450]:
%%time
with io.BytesIO(file_bytes) as file:
    batch_loaded = xr.load_dataset(file, engine='h5netcdf')

CPU times: user 201 ms, sys: 3.77 ms, total: 204 ms
Wall time: 202 ms


In [190]:
%%time
with gcs.open('gs://solar-pv-nowcasting-data/prepared_ML_training_data/test_nc4.nc', mode='rb') as file:
    batch_loaded = xr.load_dataset(file, engine='h5netcdf')

CPU times: user 415 ms, sys: 58.8 ms, total: 474 ms
Wall time: 2.22 s


## Copy file to local filesystem


Create ramdisk with:

```
sudo mkdir /mnt/ramdisk
sudo mount -t tmpfs -o size=64m tmpfs /mnt/ramdisk
sudo chown jack /mnt/ramdisk
```

In [451]:
download_location = '/mnt/ramdisk/test_nc4.nc'

In [476]:
%%time
gcs.get('gs://solar-pv-nowcasting-data/prepared_ML_training_data/test_nc4.nc', download_location)

CPU times: user 20.5 ms, sys: 15 ms, total: 35.5 ms
Wall time: 107 ms


[None]

In [477]:
%%time
batch_loaded = xr.load_dataset(download_location)

CPU times: user 133 ms, sys: 16.1 ms, total: 149 ms
Wall time: 148 ms
